In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy import reshape
import re
import ast
%matplotlib inline

In [70]:
df = pd.read_csv('gpu_parts.csv')

In [2]:
os.chdir("/Users/andrewcaffey/Documents/Projects/Data/PCPP/parts/GPU/parts/")
part_list = []
comments = []
for i in os.listdir(os.getcwd()):
    a = open(i, 'r')
    #print a.read()
    b = BeautifulSoup(a)

    #basic info
    if b.find('h4', attrs={'class':'kind'}) != None:
        kind = b.find('h4', attrs={'class':'kind'}).text
        part_name = b.find('h1', attrs={'class':'name'}).text
        link = b.find('input', attrs={'name':'url'})['value']
        info_dict = {'Kind':kind, 'Name':part_name, 'Link': link}

        #prices
        if b.find_all('td', attrs={'class':'base'}) != None:
            price_list = b.find_all('td', attrs={'class':'base'})
            price_list = [float(x.text.strip('$')) for x in price_list]
            #average_price = sum(price_list)/len(price_list)
            price_dict = {'Prices':price_list,}

        #specs
        spec_labels = b.find('div', attrs={'class':'specs block'}).find_all('h4')
        spec_labels = [x.contents[0] for x in spec_labels]
        spec_values = str(b.find('div', attrs={'class':'specs block'}))

        vals = [x.strip().split('</h4>')[1].strip('\n').strip() for x in spec_values.split("<h4>")[1:]]
        vals[-1] = vals[-1].split('\n')[0]
        spec_values = vals

        spec_values = spec_values[0:len(spec_labels)+1]
        spec_dict = {spec_label:spec_value for spec_label, spec_value in zip(spec_labels,spec_values)}

        part_dict = dict(spec_dict.items() + info_dict.items() + price_dict.items())
        part_list.append(part_dict)

        #reviews
    reviews = b.find('div', attrs={'class':'part-reviews'})
    if reviews != None:
        reviews = reviews.find_all('div',attrs={'class':'part-review-block'})
        star_list = [len(reviews[x].find('ul',attrs={'class':'stars'}).find_all('li',attrs={'class':'full-star'})) for x in range(len(reviews))]

        comment_text_list = b.find_all('div', attrs={'class':'comment-message markdown'})
        comment_text_list = [comment_text_list[x].find_all('p') for x in range(len(comment_text_list))]

        comment_text_list_clean = []
        for i, x in enumerate(comment_text_list):
            comment = ""
            for y in x:
                try:
                    comment += y.contents[0] + " "
                except:
                    pass
            comment_text_list_clean.append(comment)

        

        review = zip(star_list, comment_text_list_clean)
        
        comments.append(review)

    a.close()
    b.decompose()

In [8]:
df = pd.DataFrame(part_list)

In [9]:
clock_speed = [x.upper() if type(x)==str else '0MHZ' for x in df['Core Clock']]
clock_speed_in_mhz = [float(re.sub(r'[A-Z]+', u'', x))*1000 if 'G' in x else float(x.strip('MHZ')) for x in clock_speed]
df['clock_speed_in_mhz'] = clock_speed_in_mhz


gpu_memory_in_mb = [x.upper() for x in df['Memory Size']]
gpu_memory_in_mb = [re.sub(r'[A-Z]+', u'', x) for x in gpu_memory_in_mb]
gpu_memory_in_mb = [float(x) if x != u'N/A' else 0 for x in gpu_memory_in_mb]
df['memory_mb'] = [x * 1000 if x < 100 else x for x in gpu_memory_in_mb]

#average price
df['avg'] = [sum(x)/len(x) if len(x)>0 else 0 for x in df.Prices]

#make
df['make'] = ['NVIDIA' if 'GEFORCE' in x.upper() else 'AMD' for x in df.Chipset]

#boost clock
boost_clock_speed_in_ghz = [x.upper() if type(x)==str else '0' for x in df['Boost Clock']]
boost_clock_speed_in_ghz = [re.sub(r'[A-Z]+[A-Z]', u'', x)for x in boost_clock_speed_in_ghz]
df['boost_clock_speed_mhz'] = [float(x) if x != u'N/A' else 0 for x in boost_clock_speed_in_ghz]
df.boost_clock_speed_mhz = [x * 1000 if x < 100 else x for x in df.boost_clock_speed_mhz]

#average price
df['avg'] = [x if type(x)!= float else [0] for x in df.Prices]
df.avg = [sum(x)/len(x) if len(x)>0 else 0 for x in df.Prices]

#short_link
df['short_link'] = [x.split('/')[-1] for x in df.Link]

#TDP
df['tdp'] = [float(x.split(' ')[0].strip()) for x in df.TDP]

#length
df['gpu_length'] = [float(x.split('"')[0]) if type(x)==str else 0 for x in df.Length]

In [11]:
df.to_csv('gpu_parts.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('gpu_parts.csv')

In [ ]:
cols = ['avg', 'Manufacturer', 'Name', 'Chipset', ]

In [57]:
df.columns

Index([             Boost Clock,                  Chipset,
                     Core Clock,        CrossFire Support,
                            DVI,          DVI-D Dual-Link,
              DVI-D Single-Link,          DVI-I Dual-Link,
              DVI-I Single-Link,              DisplayPort,
                            Fan,                     HDMI,
                      Interface,                  u'Kind',
                         Length,                  u'Link',
                   Manufacturer,              Memory Size,
                    Memory Type,        Mini-Display Port,
                      Mini-HDMI,                    Model,
                        u'Name',                   Part #,
                      u'Prices',                  S-Video,
                    SLI Support,        Supports Freesync,
                Supports G-Sync,                      TDP,
                            VGA,                    VHDCI,
                   u'memory_mb',                   u'avg

In [36]:
stars = [x[0][0] for x in comments[:]]
text = [x[0][1] for x in comments[:]]
gpu_part_reviews = pd.DataFrame({'Stars':stars, 'Comments':text, 'Type':'GPU'})

#save comment dataframe
gpu_part_reviews.to_csv('gpu_part_reviews.csv', encoding='utf-8', index=False)